In [1]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Does nn.Conv2d init work well?

In [2]:
#export
from exp.nb_02 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): 
    return (x-m)/s

In [3]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [4]:
x_train, y_train, x_valid, y_valid = get_data()
train_mean, train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [7]:
x_train.shape
x_valid.shape

torch.Size([50000, 784])

torch.Size([10000, 784])

In [9]:
# x_train.view?
# the axis with -1 will be inferred after satisfying other dimensions

In [10]:
x_train = x_train.view(-1,1,28,28)
x_valid = x_valid.view(-1,1,28,28)
x_train.shape, x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [13]:
x_train[0].shape
x_train[0]

torch.Size([1, 28, 28])

tensor([[[-0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
          -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
          -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
          -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244],
         [-0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
          -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
          -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
          -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244],
         [-0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
          -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
          -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
          -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244],
         [-0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
          -0.4244, -0.4244, -0.4244, -0.4244, -0

In [14]:
x_train.shape

torch.Size([50000, 1, 28, 28])

In [15]:
n, *_ = x_train.shape
c = y_train.max()+1
nh = 32
n, c

(50000, tensor(10))

In [19]:
nn.Conv2d?

In [20]:
l1 = nn.Conv2d(1, nh, 5)

In [21]:
x = x_valid[:100]

In [22]:
x.shape

torch.Size([100, 1, 28, 28])

In [24]:
def stats(x): 
    return x.mean(),x.std()

In [25]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [31]:
l1.weight
# weights are the parameters of the filters 5x5

Parameter containing:
tensor([[[[ 1.5305e-01,  4.3973e-02, -1.8018e-01,  8.9057e-02, -6.8565e-03],
          [ 4.5147e-02,  1.5095e-01, -6.5620e-02,  1.9973e-01, -3.1179e-02],
          [-9.2781e-02, -9.3001e-02,  1.3376e-01,  1.7218e-01,  1.6238e-01],
          [-3.6836e-03,  1.5503e-01,  7.3233e-03,  1.9980e-01, -1.4442e-02],
          [-6.1292e-02, -3.2841e-02,  1.0299e-01,  1.0410e-01, -1.1255e-01]]],


        [[[-5.5599e-02, -1.7335e-01, -1.0627e-01,  9.9119e-02,  1.9064e-01],
          [ 1.6417e-01, -1.5424e-01, -8.3298e-02, -1.3486e-01, -1.8459e-01],
          [ 9.5099e-02,  7.8442e-02,  1.7950e-01, -2.8585e-02,  2.0400e-02],
          [ 9.6774e-02, -3.7884e-02, -1.3991e-01,  1.3918e-01,  1.6853e-01],
          [-4.5357e-02, -1.2213e-01, -1.9915e-02, -4.4388e-02,  1.8759e-01]]],


        [[[ 1.6768e-01, -7.6129e-02, -6.1359e-02,  2.6852e-02,  1.0443e-02],
          [-9.9161e-04,  9.7234e-02, -3.3114e-02,  5.3664e-02, -8.5302e-02],
          [ 3.9161e-02, -1.8062e-01,  1.6248e-

In [35]:
l1.weight.mean(),l1.weight.std()

(tensor(-0.0056, grad_fn=<MeanBackward1>),
 tensor(0.1147, grad_fn=<StdBackward0>))

In [32]:
stats(l1.weight),stats(l1.bias)

((tensor(-0.0056, grad_fn=<MeanBackward1>),
  tensor(0.1147, grad_fn=<StdBackward0>)),
 (tensor(-0.0262, grad_fn=<MeanBackward1>),
  tensor(0.1130, grad_fn=<StdBackward0>)))

In [36]:
# weights and bias don't have a std of 1

In [39]:
# x is normalized
x.mean(), x.std()

(tensor(-0.0362), tensor(0.9602))

In [40]:
t = l1(x)

In [41]:
stats(t)

(tensor(-0.0401, grad_fn=<MeanBackward1>),
 tensor(0.6875, grad_fn=<StdBackward0>))

In [42]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

Parameter containing:
tensor([[[[ 1.5316e-02, -4.2307e-01,  8.5438e-02,  1.5644e-02, -4.6021e-02],
          [-3.9305e-02,  2.3060e-01,  1.9322e-01, -1.9724e-01, -2.4415e-01],
          [-1.6293e-01, -2.3594e-01,  5.3319e-02, -1.4747e-01, -5.7991e-01],
          [ 2.3954e-01,  6.6165e-02, -3.3133e-01, -2.3827e-01, -8.8839e-02],
          [-4.7461e-02, -1.1625e-01, -3.0992e-01, -3.2268e-02,  2.4181e-01]]],


        [[[-2.8257e-02, -1.0772e-01,  4.4440e-01, -6.3958e-02,  5.4605e-02],
          [ 2.8746e-02,  5.0233e-02,  1.7663e-01,  8.4253e-02, -3.2134e-01],
          [ 3.5770e-01, -3.8548e-02, -1.5455e-01,  1.5949e-01,  1.4878e-02],
          [ 1.0374e-01, -1.5621e-01, -1.7566e-03,  2.4157e-01,  2.0378e-01],
          [-1.6133e-01, -4.8634e-02, -4.5946e-02, -2.9839e-01,  2.7554e-01]]],


        [[[-4.0556e-01, -2.0152e-01, -5.6900e-01, -9.5956e-02, -5.3498e-02],
          [ 1.6556e-01, -3.0047e-01, -2.6566e-02,  3.8240e-01,  4.6176e-02],
          [ 3.7710e-01,  2.8909e-01,  1.7441e-

(tensor(-0.0338, grad_fn=<MeanBackward1>),
 tensor(1.0756, grad_fn=<StdBackward0>))

In [43]:
import torch.nn.functional as F

In [44]:
def f1(x, a=0): 
    return F.leaky_relu(l1(x), a)

In [45]:
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

Parameter containing:
tensor([[[[ 3.8532e-01, -1.0358e-01, -7.8753e-02, -2.9840e-01, -4.2033e-01],
          [ 2.6819e-01,  2.6511e-01,  5.1793e-01, -2.5031e-01, -1.5346e-01],
          [ 2.0728e-01,  1.5650e-01,  4.3134e-01, -5.9355e-01,  9.4485e-02],
          [-4.1509e-01,  4.7161e-02,  3.0561e-01,  3.6979e-01,  3.0675e-02],
          [ 4.3347e-02, -5.0587e-01,  2.5271e-01, -5.3263e-02,  4.6035e-01]]],


        [[[-2.0011e-01, -1.8648e-01,  4.6228e-01, -2.4302e-01, -1.8877e-01],
          [ 4.3978e-02, -1.9394e-01,  9.1493e-02,  9.5406e-02,  1.4350e-01],
          [ 4.9728e-01,  1.7089e-01, -8.1388e-01, -2.1266e-02,  9.9728e-02],
          [ 2.6721e-01, -5.5116e-01,  3.7986e-02, -5.7862e-01, -9.8242e-02],
          [ 3.1858e-01,  2.2371e-03,  3.4273e-01,  1.6613e-01, -7.3118e-02]]],


        [[[-1.0381e-01,  1.5686e-01, -7.8645e-02, -1.5977e-01,  1.8220e-01],
          [-2.2632e-01, -7.9221e-02,  3.5652e-02,  3.8042e-01,  1.3038e-01],
          [ 4.0058e-02, -1.6622e-01,  3.2210e-

(tensor(0.4621, grad_fn=<MeanBackward1>),
 tensor(0.8627, grad_fn=<StdBackward0>))

In [46]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.2104, grad_fn=<MeanBackward1>),
 tensor(0.3718, grad_fn=<StdBackward0>))

In [47]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [49]:
l1.weight[0,0]

tensor([[ 0.1427, -0.1137,  0.1747,  0.0707, -0.0364],
        [ 0.1240,  0.0012, -0.1430, -0.0881,  0.1867],
        [ 0.1380, -0.0295,  0.1028,  0.0769, -0.0642],
        [ 0.1719, -0.0240, -0.1872, -0.1266,  0.0096],
        [-0.0731, -0.1242,  0.1947,  0.0914, -0.1481]],
       grad_fn=<SelectBackward>)

In [48]:
# receptive field size
rec_fs = l1.weight[0,0].numel()
rec_fs

25

In [50]:
nf, ni, *_ = l1.weight.shape
nf, ni

(32, 1)

In [51]:
fan_in  = ni*rec_fs
fan_out = nf*rec_fs
fan_in, fan_out, 32*25

(25, 800, 800)

In [52]:
def gain(a):
    return math.sqrt(2.0 / (1 + a**2))

In [53]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)

In [54]:
torch.zeros(10000).uniform_(-1,1).std()

tensor(0.5723)

In [55]:
1/math.sqrt(3.)

0.5773502691896258

In [56]:
def kaiming2(x, a, use_fan_out=False):
    nf, ni, *_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound)

In [57]:
kaiming2(l1.weight, a=0);
stats(f1(x))

(tensor(0.5555, grad_fn=<MeanBackward1>),
 tensor(1.0142, grad_fn=<StdBackward0>))

In [58]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.2153, grad_fn=<MeanBackward1>),
 tensor(0.4079, grad_fn=<StdBackward0>))

In [59]:
class Flatten(nn.Module):
    def forward(self,x): 
        return x.view(-1)

In [60]:
# -1 is to infer the size of the dimension
a = torch.arange(6).view(3, -1)
a
a.view(-1)

tensor([[0, 1],
        [2, 3],
        [4, 5]])

tensor([0, 1, 2, 3, 4, 5])

In [61]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [62]:
y = y_valid[:100].float()

In [63]:
t = m(x)
stats(t)

(tensor(0.0381, grad_fn=<MeanBackward1>),
 tensor(0.0074, grad_fn=<StdBackward0>))

In [64]:
l = mse(t,y)
l.backward()

In [72]:
m[0].weight.grad

tensor([[[[ 3.5537e-03,  1.8137e-02,  3.2925e-02,  2.1277e-02, -3.8492e-04],
          [ 3.0252e-03,  2.1228e-02,  3.8961e-02,  2.7443e-02, -4.2020e-05],
          [-2.5745e-04,  1.6291e-02,  3.6663e-02,  2.2758e-02, -4.3677e-03],
          [ 1.1421e-02,  1.9442e-02,  3.6071e-02,  1.8868e-02,  4.2528e-05],
          [ 2.3984e-02,  3.0115e-02,  2.7490e-02,  1.9368e-02,  7.9593e-03]]],


        [[[ 4.9405e-02,  3.6284e-02,  9.4223e-03, -8.8381e-03, -1.8642e-02],
          [ 5.5951e-02,  3.7077e-02, -2.0950e-03, -1.7608e-02, -9.0943e-03],
          [ 7.1347e-02,  4.4489e-02,  1.1819e-02,  8.3565e-03,  9.4747e-03],
          [ 7.1265e-02,  5.1793e-02,  2.9635e-02,  2.5590e-02,  1.9192e-02],
          [ 6.2188e-02,  3.6877e-02,  2.2241e-02,  2.1637e-02,  1.7116e-02]]],


        [[[-1.0344e-02, -6.2745e-03, -2.8135e-03,  4.4970e-03, -5.4039e-03],
          [-1.0517e-02, -3.0707e-03,  1.9864e-03, -4.0595e-04, -1.8014e-02],
          [-1.3695e-02, -1.9094e-03,  4.9194e-03, -1.0149e-02, -2.62

In [65]:
stats(m[0].weight.grad)

(tensor(0.0175), tensor(0.0214))

In [39]:
init.kaiming_uniform_??

In [73]:
for l in m:
    if isinstance(l, nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

Parameter containing:
tensor([[[[-0.0944, -0.0692, -0.1382,  0.0765, -0.0391],
          [-0.3933, -0.0231,  0.0357, -0.3811, -0.0820],
          [-0.3234, -0.2863, -0.0598, -0.3834, -0.1741],
          [ 0.3534,  0.4898,  0.2462,  0.4087,  0.1174],
          [ 0.0166,  0.2811, -0.0730, -0.3248, -0.0186]]],


        [[[-0.3488, -0.3042,  0.0357, -0.1347, -0.2351],
          [ 0.2602, -0.0887, -0.1875, -0.2744, -0.4780],
          [-0.2020,  0.0556,  0.1108, -0.3842,  0.2458],
          [ 0.3966,  0.2926, -0.2597,  0.3940,  0.3470],
          [-0.0494, -0.3532, -0.4360,  0.3422, -0.4211]]],


        [[[-0.3002,  0.1897, -0.2394, -0.2862, -0.1821],
          [ 0.2979, -0.2530, -0.3640, -0.3552, -0.4483],
          [-0.4392,  0.3830,  0.4675, -0.0614, -0.1898],
          [ 0.2728,  0.3942,  0.4003,  0.4508,  0.2322],
          [ 0.3821, -0.1841, -0.0347, -0.1091,  0.0280]]],


        [[[ 0.3151,  0.0204,  0.1226,  0.1059, -0.0538],
          [ 0.2265,  0.2168, -0.1657,  0.0847, -0.0134

tensor([0., 0., 0., 0., 0., 0., 0., 0.])

Parameter containing:
tensor([[[[ 0.1870, -0.1263, -0.0004],
          [-0.1131, -0.2550,  0.2809],
          [-0.2015, -0.2858,  0.0638]],

         [[ 0.1562, -0.1016,  0.2549],
          [-0.1209,  0.1070, -0.0278],
          [-0.1201,  0.1860,  0.0828]],

         [[ 0.0927,  0.2487, -0.0547],
          [-0.0550,  0.0635, -0.2612],
          [-0.0745,  0.1620,  0.1840]],

         ...,

         [[ 0.0834,  0.0035, -0.2730],
          [ 0.1492,  0.1522, -0.0371],
          [-0.0243, -0.2399,  0.1303]],

         [[-0.0070,  0.1817,  0.0382],
          [ 0.2209,  0.0588, -0.0214],
          [ 0.1641, -0.1322,  0.0912]],

         [[-0.0177,  0.1710, -0.2705],
          [-0.2111, -0.1069,  0.2690],
          [-0.1816, -0.1621,  0.2050]]],


        [[[ 0.1595,  0.0831, -0.2457],
          [ 0.1682,  0.0113, -0.0461],
          [ 0.0632, -0.1071, -0.2211]],

         [[-0.2066,  0.1512, -0.0655],
          [ 0.0585, -0.2334,  0.0555],
          [-0.2712,  0.2329,  0.1304]],

         

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Parameter containing:
tensor([[[[ 0.0443,  0.1846, -0.0385],
          [-0.0671,  0.0212,  0.0347],
          [ 0.1541,  0.0615, -0.0216]],

         [[ 0.1385,  0.0225, -0.0039],
          [-0.0575, -0.1782,  0.1012],
          [-0.1817,  0.0129, -0.2018]],

         [[ 0.0343,  0.1231,  0.1745],
          [-0.1716,  0.0768,  0.0699],
          [-0.0342, -0.0430, -0.0766]],

         ...,

         [[ 0.0932,  0.1593, -0.0739],
          [ 0.1830, -0.0133, -0.0096],
          [-0.1293,  0.1397,  0.1128]],

         [[-0.0109, -0.1436, -0.1397],
          [-0.1277, -0.1258,  0.0995],
          [ 0.1137,  0.0037, -0.1208]],

         [[-0.1890,  0.0628, -0.0082],
          [ 0.2006,  0.1886, -0.0294],
          [-0.1475,  0.0271,  0.1092]]],


        [[[ 0.0398,  0.0456,  0.0341],
          [-0.0046,  0.0999,  0.0762],
          [-0.0809,  0.1816,  0.0036]],

         [[ 0.0101, -0.1565, -0.0049],
          [ 0.1720,  0.1377, -0.1422],
          [ 0.1242,  0.1500, -0.0543]],

         

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])

Parameter containing:
tensor([[[[ 0.0649, -0.1401, -0.1082],
          [ 0.0467,  0.0663, -0.0909],
          [-0.0844,  0.0894, -0.1126]],

         [[ 0.1349,  0.0177, -0.0914],
          [ 0.0418,  0.1142,  0.0668],
          [ 0.0433,  0.1038, -0.0816]],

         [[-0.0854, -0.0813,  0.0970],
          [ 0.0292,  0.0910, -0.0250],
          [ 0.0702, -0.1375,  0.0547]],

         [[-0.0067,  0.0669, -0.0292],
          [ 0.0553, -0.0814,  0.0081],
          [-0.0099,  0.0262,  0.0605]],

         [[-0.1262,  0.1407, -0.1108],
          [ 0.0852,  0.0647, -0.1207],
          [ 0.1405,  0.0453, -0.0864]],

         [[-0.0524, -0.0040,  0.1111],
          [-0.0865, -0.1414,  0.0075],
          [-0.0605, -0.0864,  0.0344]],

         [[ 0.0409, -0.0377,  0.1258],
          [ 0.0646, -0.0909, -0.0334],
          [ 0.0862,  0.0974, -0.0335]],

         [[ 0.0367, -0.0135, -0.0179],
          [-0.0940, -0.1390, -0.0525],
          [-0.0526, -0.1028, -0.0877]],

         [[-0.0133,  0.124

tensor([0.])

In [74]:
t = m(x)
stats(t)

(tensor(0.4555, grad_fn=<MeanBackward1>),
 tensor(0.3109, grad_fn=<StdBackward0>))

In [75]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(-0.3008), tensor(0.5359))

## Export

In [ ]:
!./notebook2script.py 02a_why_sqrt5.ipynb